# PERSIPAN PUSTAKA

library dipakai:
1. selenium
2. beautifulsoup4
3. paddleOCR
4. cv2
5. pandas
6. geopandas
7. shapely
8. matplotlib

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from matplotlib import pyplot as plt
import cv2
from paddleocr import PaddleOCR, draw_ocr
import cv2
from matplotlib import pyplot as plt

Alur:
1. buat screenshot layar
2. dapatkan nama tempat dengan ocr, sekaligus koordinat pada layar
3. hapus teks yang merupakan menu dari google maps
4. hapus teks yang berupa nama jalan
5. klik setiap nama tempat yang didapat
6. dapatkan koordinat POI melalui link
7. dapatkan detail dari tempat menggunakan bs4

# CONFIG PARAMETER

In [ ]:
#path shp file

#pilih kelurahan

#folder output
path_output = "output/"

# CONFIG SELENIUM DRIVER

In [ ]:
#windows option
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_experimental_option("detach",True)

# SCREENSHOOT

In [3]:
def make_screenshot(driver:webdriver.Chrome, link:str, save_to:str):
    driver.get(link)
    driver.save_screenshot(save_to)

    #windows size
    print(driver.get_window_size())

# OCR

In [ ]:
# Initialize OCR
ocr = PaddleOCR(use_angle_cls= True, lang = "id")

# Path
image_path = "./test.png"
font_path = "C:/Windows/Fonts/arial.ttf"

# Perform OCR
result = ocr.ocr(image_path, cls=True)

# Load the image
image = cv2.imread(image_path)

# Extract
for line in result:
    for word in line:
        print(f"Detected text: {word[1][0]}, confidence: {word[1][1]}")

# Visualization
# Get the bounding box and corresponding text
boxes = [line[0] for line in result[0]]
txts = [line[1][0] for line in result[0]]
scores = [line[1][1] for line in result[0]]

# Draw the result on the image
image_with_boxes = draw_ocr(image, boxes, font_path = font_path)

# Convert the image from BGR to RGB (OpenCV loads images in BGR)
image_with_boxes = cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB)

plt.imshow(image_with_boxes)
plt.show()

# GET COORDINATE

In [ ]:
tag = "!4m6!3m5!1s0x2e68498154b496b1:0x8ce79800c09324a2!8m2!3d-6.9199873!4d106.9417309!16s%2Fg%2F11s7w8jwvg?entry=ttu&g_ep=EgoyMDI0MTAyOS4wIKXMDSoASAFQAw%3D%3D"

# EXCEXUTE

In [ ]:
#initiate browser
driver = webdriver.Chrome(options=chrome_options)

#looping

#
make_screenshot(driver, "https://google.com","ss.png")

# driver.quit()

{'width': 1936, 'height': 1048}
